<a href="https://colab.research.google.com/github/LC1332/Luotuo-Chinese-LLM/blob/main/notebook/improvedTranslation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 一个升级后的批量翻译代码

这个代码最初由黄泓森进行开发，由李鲁鲁转到colab并进行了更改

[骆驼项目主页](https://github.com/LC1332/Luotuo-Chinese-LLM)

如果你使用我们的代码获取了有用的数据，也欢迎分享给我们，或者告诉我们你公开后的github/huggingface链接

如果你使用我们的代码获取数据并发表了论文或者tech report，欢迎cite我们的github repo

## 安装环境

In [ ]:
!pip install openai
!pip install aiofiles

In [2]:
import os
import json
import time
import openai
import asyncio
import aiohttp
import aiofiles
from functools import partial
from tqdm.asyncio import tqdm as tqdm

## 输入你的openAI API

In [3]:
# 在这里输入你的openAI API token

api_key = ["sk-r4"]


class KeyPool:
    def __init__(self, strings):
        self.pool = list(strings)
        self.last_used = {s: -1 for s in strings}

    def getKey(self):
        result = min(self.last_used, key=self.last_used.get)
        self.last_used[result] = int(time.time() * 1000)
        return result

pool = KeyPool(api_key)

## 指定工作目录



In [4]:
os.chdir("/content/")

## 获取需要翻译的样本

这里我们使用WizardLM的样本

In [5]:
!wget https://raw.githubusercontent.com/LC1332/WizardLM/main/data/WizardLM_testset.jsonl -O WizardLM_testset.jsonl

--2023-05-10 05:32:03--  https://raw.githubusercontent.com/LC1332/WizardLM/main/data/WizardLM_testset.jsonl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 81139 (79K) [text/plain]
Saving to: ‘WizardLM_testset.jsonl’

WizardLM_testset.js 100%[===================>]  79.24K  --.-KB/s    in 0.004s  

2023-05-10 05:32:03 (17.7 MB/s) - ‘WizardLM_testset.jsonl’ saved [81139/81139]



In [6]:
delay = 0.05

concurrency_limit = 64

input_file = "WizardLM_testset.jsonl"

# 数据缓存目录
temp_path = "/content/temp"

# 数据输出目录
output_path = "/content/translate"

output_prefix = "WizardLM_tr"

max_file_size = 1024**3

# 需要翻译的字段
entries = ["Instruction"]

os.system(f"mkdir -p {temp_path} {output_path}")

0

In [ ]:
# previous message
# [
#                     {
#                         "role": "system",
#                         "content": "这是一个能够将文本翻译成中文的AI助手。请将引号中的文本翻译成简体中文。",
#                     },
#                     {"role": "user", "content": f'"""\n{text}\n"""'},
#                 ]

In [17]:
import re
my_str = '```ResNet主要利用了形如f(x)+x的残差Block，使得反向传播可以更加顺利。```'

# my_str = resp['choices'][0]['message']['content']
pattern = r'```([^\s]+)```'
# 搜索匹配的结果
match_res = re.search(pattern, my_str)

# 输出第一组反引号中间的内容
if match_res:
    print( match_res.group(1) )
else:
    print( my_str )

ResNet主要利用了形如f(x)+x的残差Block，使得反向传播可以更加顺利。


In [18]:
print(len(my_str),len(match_res.group(1) ))
# print

47 41


<function print>

In [21]:
import re

async def getTranslation(item, entries: list = []):
    async def get(text):
        text = text.replace("\n", " ")
        openai.api_key = pool.getKey()
        try:
            messages =  [  
            {'role':'system', 'content':'你是一个能够将文本翻译成中文的AI助手。请将反引号中的英文文本翻译成简体中文。'},    
            {'role':'user', 'content':'将反引号中的指令翻译成中文:```ResNet mainly utilizes residual blocks like f(x)+x, which makes the backpropagation smoother```'},   
            {'role':'assistant', 'content':'```ResNet主要利用了形如f(x)+x的残差Block，使得反向传播可以更加顺利。```'},   
            {'role':'user', 'content':'将反引号中的指令翻译成中文:```Who are you?```'},
            {'role':'assistant', 'content':'```你是谁?```'},
            {'role':'user', 'content':f'将反引号中的指令翻译成中文:```{text}```'}  ]

            resp = await openai.ChatCompletion.acreate(
                model="gpt-3.5-turbo",
                messages=messages,
                temperature=0.3,
                max_tokens=100,
                top_p=1.0,
                frequency_penalty=0.0,
                presence_penalty=0.0,
            )
            if "choices" in resp:
                my_str = resp['choices'][0]['message']['content']

                pattern = r'```([^\s]+)```'
                # 搜索匹配的结果
                match_res = re.search(pattern, my_str)
                
                # 输出第一组反引号中间的内容
                if match_res and (len(my_str) - len(match_res) == 6 or len(my_str) - len(match_res) == 7):
                    return match_res.group(1)
                else:
                    return my_str
            else:
                raise Exception(f"Invalid API response: {resp}")
        except Exception as e:
            print(f"[Error] {e}")
            return None

    for entry in entries:
        trans = await get(item[entry])
        if trans is None:
            return None
        else:
            item[f"{entry}_zh"] = trans
    return item


async def process(id, item, semaphore):
    async with semaphore:
        file_name = f"{temp_path}/{output_prefix}_{id}.json"
        try:
            it = await getTranslation(item, entries)
            if it is None:
                raise Exception(file_name)
            async with aiofiles.open(file_name, "w") as f:
                await f.write(json.dumps(it, ensure_ascii=False, indent=4))
        except Exception as e:
            print(f"Error saving item: {e}")


async def main():
    try:
      with open(input_file, "r") as file:
          data = json.load(file)
    except json.JSONDecodeError:
      data = []
      with open(input_file, "r") as file:
          for line in file:
              entry = json.loads(line)
              data.append(entry)

    tasks = []

    semaphore = asyncio.Semaphore(concurrency_limit)

    for id, item in enumerate(data):
        if os.path.exists(f"{output_prefix}{id}.json"):
            continue
        tasks.append(asyncio.create_task(process(id, item, semaphore)))

    async for task in tqdm(tasks, total=len(tasks), desc="Processing items"):
        await task
        time.sleep(delay)

由于网络问题或OpenAI的限制会导致获取数据失败，此时脚本会跳过这部分数据

重新运行下面的单元格即可补充获取失败的数据

In [20]:
await main()

Processing items:  72%|███████▏  | 157/218 [00:40<02:16,  2.24s/it]

[Error] That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID 6fd1f3ad3de932fb53c97a4c681e5b8e in your message.)
Error saving item: /content/temp/WizardLM_tr_153.json


Processing items:  97%|█████████▋| 211/218 [00:45<00:02,  2.64it/s]

[Error] That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID 755eb5a662aba962501d01b7b4cd0bf3 in your message.)
Error saving item: /content/temp/WizardLM_tr_208.json


Processing items: 100%|██████████| 218/218 [00:45<00:00,  4.78it/s]


## 合并所有翻译数据

In [14]:
data = []
for filename in tqdm(os.listdir(temp_path)):
    if filename.startswith(output_prefix) and filename.endswith(".json"):
        with open(os.path.join(temp_path, filename), 'r', encoding='utf-8') as file:
            try:
                entry = json.load(file)
                data.append(entry)
            except json.JSONDecodeError:
                pass

100%|██████████| 212/212 [00:00<00:00, 16413.03it/s]


In [15]:
file_counter = 1
current_file_size = 0
output_file = f"{output_path}/{output_prefix}_{file_counter}.jsonl"

with open(output_file, 'w', encoding='utf-8') as out:
    for item in tqdm(data):
        item_json = json.dumps(item, ensure_ascii=False)
        item_size = len(item_json.encode('utf-8'))
        out.write(item_json + "\n")
        current_file_size += item_size
        if current_file_size > max_file_size:
            file_counter += 1
            output_file = f"{output_path}/{output_prefix}_{file_counter}.jsonl"
            out = open(output_file, 'w', encoding='utf-8')
            current_file_size = 0

100%|██████████| 212/212 [00:00<00:00, 46520.48it/s]


In [16]:
print(output_file)

/content/translate/WizardLM_tr_1.jsonl
